# Qassida : un générateur de poésie arabe avec LSTM sous Pytorch


*   A rédiger : Membres de l'équipe
*   Introduction + problématique 
*   Etapes de la réalistion



### Import des librairies :     
Nous commençons par l'import des librairies dont nous aurons besoin pour pouvoir implémenter notre solution
> * NumPy pour la gestion des calculs matriciels et vectoriels
> * PyTorch permet d'effectuer les calculs tensoriels nécessaires notamment pour l'apprentissage profond
> * Beautiful Soup pour le web scraping et la génération du dataset  

In [ ]:
import torch
from torch import nn
import torch.nn.functional as func
import numpy as np

### Chargement des données
Initialement nous exécutons un programme de web scraping qui fournit les données sous forme d'un fichier texte, pour effectuer l'entrainement après.

> Nous chargeons d'abord, le résultat du scraping à partir du fichier texte pour le pré-traiter 

In [ ]:
# Ouvrir le fichier texte lire les données sous forme de 'texte'

with open('poems_of_Darwish.txt', 'r', encoding="utf-8") as f:
  poem = f.read();

> Visualisons les 50 premiers caractères du texte :

In [ ]:
poem[:50]

'علي شاطء البحر بنت  وللبنت اهل\nوللاهل بيت  وللبيت '

### Vectorisation
Dans ce qui suit, nous utilisons les structures 'dictionaries' de python pour convertir les caractères en un entier unique

In [ ]:
# Fonction 1 : char_to_int 
# Fonction 2 : int_to_char 

vocab = tuple(set(poem))

int_to_char = dict(enumerate(vocab))
char_to_int = {char: ii for ii, char in int_to_char.items()}

encoded = np.array([char_to_int[char] for char in poem])

> Visualisons l'encodage des 50 premiers caractères

In [ ]:
encoded[:50]

array([46, 55, 45, 42, 26, 53, 66, 47, 42, 53, 55, 14,  3,  4, 42, 14, 54,
       16, 42, 42, 49, 55, 55, 14, 54, 16, 42, 53,  6, 55, 17, 49, 55, 55,
       53,  6, 55, 42, 14, 45, 16, 42, 42, 49, 55, 55, 14, 45, 16, 42])

### Partie pré-traitement des données

Dans le modèle charRNN que nous voulant utiliser, le LSTM prend en entrée des données one-hot-encoded 
i.e un vecteur dont la taille est celle du vocabulaire et pour chaque caractère un '1' est placé à
la position du caractère dans ce vecteur, sinon '0'

In [ ]:
def one_hot(vect, taille_vocab):
  one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
  one_hot = one_hot.reshape((*arr.shape, n_labels))
    
  return one_hot

### Division en mini-batches d'entrainement

> On divise le vecteur des caractères encodés en séquence de taile seq_length selon le batch_size.


### Créatio des batches
> 1 - D'abord, il faut retrancher certain caractères pour avoir des batches de taille complète uniquement.
> 2 - Diviser le vecteur de caractères en N batches.
> 3 - Parcourir le vecteur qui est maintenant divisé en N sous-vecteurs pour avoir les mini-batches. 

### Entrainer et tester les Batched
> 1 - Créer 2 batches x,y où : x est le input batch et y est le training batch qui est exactement x mais décalé d'un seul caractère.

In [ ]:
def get_batches(arr, batch_size, seq_length):

    batch_size_total = batch_size * seq_length

    # Nombre total de batches que nous pouvons avoir
    n_batches = len(arr)//batch_size_total
    
    # Garder que les batches complets 
    arr = arr[:n_batches * batch_size_total]

    # Reshape en lignes de batch_size 
    arr = arr.reshape((batch_size, -1))
    
    # Itérer sur chaque séquence de caractères
    for n in range(0, arr.shape[1], seq_length):

        x = arr[:, n:n+seq_length]
        
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Visualiser la sortie 
> Visualisons ce que les batches sur 100 caractères de données encodées donne :

In [ ]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

# Les 10 premiers éléments d'une séquence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[46 55 45 42 26 53 66 47 42 53]
 [53 42 53 55 30 53  6  4 45 54]
 [35 42 53 55 49 27 16 42 60 45]
 [17 49 30  4 54 53 42 53 55 45]
 [42  1 54 42 53 55  4 49  3 42]
 [49 42 55 22  6 17 53 55  1 42]
 [53 42  1 53 47  6 53 42 45  4]
 [54 49 53 46 42 53 55  3 18 53]]

y
 [[55 45 42 26 53 66 47 42 53 55]
 [42 53 55 30 53  6  4 45 54 42]
 [42 53 55 49 27 16 42 60 45 42]
 [49 30  4 54 53 42 53 55 45 42]
 [ 1 54 42 53 55  4 49  3 42 49]
 [42 55 22  6 17 53 55  1 42 16]
 [42  1 53 47  6 53 42 45  4 49]
 [49 53 46 42 53 55  3 18 53  4]]
